# Pytorch를 이용한 간단한 CNN 구현

## 1. 필요한 라이브러리 불러오기

In [1]:
import torch    # Pytorch 기본 라이브러리
import torch.nn as nn   # Neural Network (nn)에 관련된 모듈들
from torch.utils.data import DataLoader # Database의 데이터들을 불러오기 위한 라이브러리

from torchvision import transforms, datasets    # 불러온 입력을 처리하는 모듈들

import os   # 경로 탐색, 접근 등을 처리하기 위한 라이브러리
import numpy as np # 행렬 연산에 사용하는 라이브러리

## 2. 학습을 위한 하이퍼 파리미터 설정

In [2]:
lr = 0.01
batch_size = 128
num_epoch = 5

ckpt_dir = './checkpoint'   # 학습된 파라미터를 저장할 경로

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
print(device)

cpu


## 3. CNN 구축

In [5]:
class Net(nn.Module):       # nn.Module를 상속받아서 사용.
    def __init__(self):
        super(Net, self).__init__() # 상속해준 부모 클래스의 Method를 사용할 때 super(하위클래스, 하위클래스의 객체)

        # Block 1
        self.conv1 = nn.Conv2d(in_channels=1, 
                               out_channels=10, 
                               kernel_size=5, 
                               stride=1, 
                               padding=0)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)    # cf. AvgPool2d

        # Block 2
        self.conv2 = nn.Conv2d(in_channels=10, 
                               out_channels=20, 
                               kernel_size=5, 
                               stride=1, 
                               padding=0)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)    # H * W * C = 320

        self.fc1 = nn.Linear(in_features=320, 
                             out_features=50)             # Fully-Connected Layer
        self.relu1_fc1 = nn.ReLU()

        self.fc2 = nn.Linear(in_features=50, out_features=10, bias=True)

    def forward(self, x):   # x: 입력영상 (torch.Tensor())
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = x.view(-1, 320)     # x: B=128, C=20, H=4, W=4

        x = self.fc1(x)
        x = self.relu1_fc1(x)

        x = self.fc2(x)

        return x

## 4. 학습된 파라미터를 저장하거나 불러오는 함수 작성

In [6]:
def save(ckpt_dir, net, optim, epoch):      # ckpt_dir: checkpoint를 저장할 경로, net, optim, epoch
    if not os.path.exists(ckpt_dir):    # ckpt_dir이 존재하는지 확인하는 함수
        os.makedirs(ckpt_dir)           # 디렉토리를 만들어주는 함수

    torch.save({'net': net.state_dict(),        # 네트워크에 있는 변수들
                'optim': optim.state_dict()},   # optimizer에 있는 변수들
               './%s/model_epoch%d.pth' % (ckpt_dir, epoch))

def load(ckpt_dir, net, optim):
    ckpt_lst = os.listdir(ckpt_dir)  # 입력한 디렉토리 내의 모든 파일과 디렉토리의 리스트를 반환
    ckpt_lst.sort()  # 정렬

    dict_model = torch.load('./%s/%s' % (ckpt_dir, ckpt_lst[-1]))

    net.load_state_dict(dict_model['net'])
    optim.load_state_dict(dict_model['optim'])

    return net, optim

서식문자를 이용한 문자열 표현

In [7]:
print('%s is %s.' % ('Cat', 'animal'))
print('%d + %d is %d' % (2, 3, 5))

Cat is animal.
2 + 3 is 5


In [8]:
a = ['b', 'a', 'c']
a.sort()
print(a)

['a', 'b', 'c']


## 5. Database 불러오기
### Database가 없을 경우 Pytorch에 내장되어 있는 MNIST 다운로드 함수 사용

In [9]:
transform = transforms.Compose(
    [transforms.ToTensor(), 
     transforms.Normalize(mean=(0.5,), std=(0.5,))])

dataset = datasets.MNIST(download=True, 
                         root='./', 
                         train=True, 
                         transform=transform)   # 개별 데이터 파일의 처리를 담당
loader = DataLoader(dataset, 
                    batch_size=batch_size, 
                    shuffle=True, 
                    num_workers=0)  # 미니배치 처리를 담당

num_data = len(loader.dataset)
num_batch = num_data // batch_size

31.0%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

83.6%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100.0%


Extracting ./MNIST\raw\train-images-idx3-ubyte.gz to ./MNIST\raw


102.8%


Extracting ./MNIST\raw\train-labels-idx1-ubyte.gz to ./MNIST\raw




25.3%

100.0%


Extracting ./MNIST\raw\t10k-images-idx3-ubyte.gz to ./MNIST\raw



112.7%

Extracting ./MNIST\raw\t10k-labels-idx1-ubyte.gz to ./MNIST\raw



In [10]:
print(num_data)

60000


## 6. 네트워크 및 손실함수 선언

In [11]:
net = Net()
net = net.to(device)
params = net.parameters()

fn_loss = nn.CrossEntropyLoss().to(device)

def fn_pred(output):
    return torch.softmax(output, dim=1)     # B=128 (dim=0), 10 (dim=1)

def fn_accuracy(pred, label):
    return (pred.max(dim=1)[1] == label).type(torch.float).mean()

## 7. Optimizer 선언

In [12]:
optimizer = torch.optim.SGD(params, lr=lr)

## 8. 학습 진행

In [13]:
for epoch in range(1, num_epoch + 1):
    net.train() # net.eval()

    loss_arr = []
    acc_arr = []

    for batch, (input, label) in enumerate(loader):
        input = input.to(device)
        label = label.to(device)

        output = net(input)     # net.forward(input)
        pred = fn_pred(output)  # softmax

        optimizer.zero_grad()   # G = 0 

        loss = fn_loss(output, label)   # 
        acc = fn_accuracy(pred, label)

        loss.backward() # 그라디언트를 계산

        optimizer.step()    # 그라디언트를 사용하여 파라미터를 업데이트

        loss_arr += [loss.item()]
        acc_arr += [acc.item()]

        print('TRAIN: EPOCH %04d/%04d | BATCH %04d/%04d | LOSS: %.4f | ACC %.4f' %
              (epoch, num_epoch, batch, num_batch, np.mean(loss_arr), np.mean(acc_arr)))

    save(ckpt_dir = ckpt_dir, net=net, optim=optimizer, epoch=epoch)

TRAIN: EPOCH 0001/0005 | BATCH 0000/0468 | LOSS: 2.3044 | ACC 0.1172
TRAIN: EPOCH 0001/0005 | BATCH 0001/0468 | LOSS: 2.3026 | ACC 0.1172
TRAIN: EPOCH 0001/0005 | BATCH 0002/0468 | LOSS: 2.3030 | ACC 0.1224
TRAIN: EPOCH 0001/0005 | BATCH 0003/0468 | LOSS: 2.3020 | ACC 0.1484
TRAIN: EPOCH 0001/0005 | BATCH 0004/0468 | LOSS: 2.3016 | ACC 0.1578
TRAIN: EPOCH 0001/0005 | BATCH 0005/0468 | LOSS: 2.3022 | ACC 0.1523
TRAIN: EPOCH 0001/0005 | BATCH 0006/0468 | LOSS: 2.3023 | ACC 0.1551
TRAIN: EPOCH 0001/0005 | BATCH 0007/0468 | LOSS: 2.3014 | ACC 0.1562
TRAIN: EPOCH 0001/0005 | BATCH 0008/0468 | LOSS: 2.3015 | ACC 0.1528
TRAIN: EPOCH 0001/0005 | BATCH 0009/0468 | LOSS: 2.3013 | ACC 0.1547
TRAIN: EPOCH 0001/0005 | BATCH 0010/0468 | LOSS: 2.3017 | ACC 0.1577
TRAIN: EPOCH 0001/0005 | BATCH 0011/0468 | LOSS: 2.3015 | ACC 0.1589
TRAIN: EPOCH 0001/0005 | BATCH 0012/0468 | LOSS: 2.3013 | ACC 0.1593
TRAIN: EPOCH 0001/0005 | BATCH 0013/0468 | LOSS: 2.3011 | ACC 0.1641
TRAIN: EPOCH 0001/0005 | BATCH 001

TRAIN: EPOCH 0001/0005 | BATCH 0120/0468 | LOSS: 2.2828 | ACC 0.2241
TRAIN: EPOCH 0001/0005 | BATCH 0121/0468 | LOSS: 2.2826 | ACC 0.2246
TRAIN: EPOCH 0001/0005 | BATCH 0122/0468 | LOSS: 2.2823 | ACC 0.2250
TRAIN: EPOCH 0001/0005 | BATCH 0123/0468 | LOSS: 2.2821 | ACC 0.2257
TRAIN: EPOCH 0001/0005 | BATCH 0124/0468 | LOSS: 2.2820 | ACC 0.2259
TRAIN: EPOCH 0001/0005 | BATCH 0125/0468 | LOSS: 2.2818 | ACC 0.2263
TRAIN: EPOCH 0001/0005 | BATCH 0126/0468 | LOSS: 2.2817 | ACC 0.2266
TRAIN: EPOCH 0001/0005 | BATCH 0127/0468 | LOSS: 2.2815 | ACC 0.2270
TRAIN: EPOCH 0001/0005 | BATCH 0128/0468 | LOSS: 2.2813 | ACC 0.2273
TRAIN: EPOCH 0001/0005 | BATCH 0129/0468 | LOSS: 2.2811 | ACC 0.2276
TRAIN: EPOCH 0001/0005 | BATCH 0130/0468 | LOSS: 2.2809 | ACC 0.2281
TRAIN: EPOCH 0001/0005 | BATCH 0131/0468 | LOSS: 2.2808 | ACC 0.2286
TRAIN: EPOCH 0001/0005 | BATCH 0132/0468 | LOSS: 2.2805 | ACC 0.2296
TRAIN: EPOCH 0001/0005 | BATCH 0133/0468 | LOSS: 2.2802 | ACC 0.2299
TRAIN: EPOCH 0001/0005 | BATCH 013

TRAIN: EPOCH 0001/0005 | BATCH 0242/0468 | LOSS: 2.2435 | ACC 0.3164
TRAIN: EPOCH 0001/0005 | BATCH 0243/0468 | LOSS: 2.2428 | ACC 0.3175
TRAIN: EPOCH 0001/0005 | BATCH 0244/0468 | LOSS: 2.2424 | ACC 0.3178
TRAIN: EPOCH 0001/0005 | BATCH 0245/0468 | LOSS: 2.2418 | ACC 0.3182
TRAIN: EPOCH 0001/0005 | BATCH 0246/0468 | LOSS: 2.2414 | ACC 0.3184
TRAIN: EPOCH 0001/0005 | BATCH 0247/0468 | LOSS: 2.2408 | ACC 0.3189
TRAIN: EPOCH 0001/0005 | BATCH 0248/0468 | LOSS: 2.2402 | ACC 0.3194
TRAIN: EPOCH 0001/0005 | BATCH 0249/0468 | LOSS: 2.2396 | ACC 0.3200
TRAIN: EPOCH 0001/0005 | BATCH 0250/0468 | LOSS: 2.2391 | ACC 0.3203
TRAIN: EPOCH 0001/0005 | BATCH 0251/0468 | LOSS: 2.2386 | ACC 0.3205
TRAIN: EPOCH 0001/0005 | BATCH 0252/0468 | LOSS: 2.2381 | ACC 0.3209
TRAIN: EPOCH 0001/0005 | BATCH 0253/0468 | LOSS: 2.2375 | ACC 0.3216
TRAIN: EPOCH 0001/0005 | BATCH 0254/0468 | LOSS: 2.2368 | ACC 0.3222
TRAIN: EPOCH 0001/0005 | BATCH 0255/0468 | LOSS: 2.2361 | ACC 0.3229
TRAIN: EPOCH 0001/0005 | BATCH 025

TRAIN: EPOCH 0001/0005 | BATCH 0365/0468 | LOSS: 2.1098 | ACC 0.3625
TRAIN: EPOCH 0001/0005 | BATCH 0366/0468 | LOSS: 2.1078 | ACC 0.3631
TRAIN: EPOCH 0001/0005 | BATCH 0367/0468 | LOSS: 2.1062 | ACC 0.3635
TRAIN: EPOCH 0001/0005 | BATCH 0368/0468 | LOSS: 2.1043 | ACC 0.3641
TRAIN: EPOCH 0001/0005 | BATCH 0369/0468 | LOSS: 2.1026 | ACC 0.3645
TRAIN: EPOCH 0001/0005 | BATCH 0370/0468 | LOSS: 2.1010 | ACC 0.3650
TRAIN: EPOCH 0001/0005 | BATCH 0371/0468 | LOSS: 2.0989 | ACC 0.3657
TRAIN: EPOCH 0001/0005 | BATCH 0372/0468 | LOSS: 2.0973 | ACC 0.3661
TRAIN: EPOCH 0001/0005 | BATCH 0373/0468 | LOSS: 2.0956 | ACC 0.3665
TRAIN: EPOCH 0001/0005 | BATCH 0374/0468 | LOSS: 2.0941 | ACC 0.3668
TRAIN: EPOCH 0001/0005 | BATCH 0375/0468 | LOSS: 2.0924 | ACC 0.3672
TRAIN: EPOCH 0001/0005 | BATCH 0376/0468 | LOSS: 2.0909 | ACC 0.3677
TRAIN: EPOCH 0001/0005 | BATCH 0377/0468 | LOSS: 2.0891 | ACC 0.3682
TRAIN: EPOCH 0001/0005 | BATCH 0378/0468 | LOSS: 2.0872 | ACC 0.3689
TRAIN: EPOCH 0001/0005 | BATCH 037

TRAIN: EPOCH 0002/0005 | BATCH 0017/0468 | LOSS: 0.9299 | ACC 0.7352
TRAIN: EPOCH 0002/0005 | BATCH 0018/0468 | LOSS: 0.9271 | ACC 0.7360
TRAIN: EPOCH 0002/0005 | BATCH 0019/0468 | LOSS: 0.9254 | ACC 0.7352
TRAIN: EPOCH 0002/0005 | BATCH 0020/0468 | LOSS: 0.9206 | ACC 0.7370
TRAIN: EPOCH 0002/0005 | BATCH 0021/0468 | LOSS: 0.9196 | ACC 0.7376
TRAIN: EPOCH 0002/0005 | BATCH 0022/0468 | LOSS: 0.9208 | ACC 0.7371
TRAIN: EPOCH 0002/0005 | BATCH 0023/0468 | LOSS: 0.9184 | ACC 0.7367
TRAIN: EPOCH 0002/0005 | BATCH 0024/0468 | LOSS: 0.9135 | ACC 0.7381
TRAIN: EPOCH 0002/0005 | BATCH 0025/0468 | LOSS: 0.9132 | ACC 0.7377
TRAIN: EPOCH 0002/0005 | BATCH 0026/0468 | LOSS: 0.9109 | ACC 0.7378
TRAIN: EPOCH 0002/0005 | BATCH 0027/0468 | LOSS: 0.9116 | ACC 0.7374
TRAIN: EPOCH 0002/0005 | BATCH 0028/0468 | LOSS: 0.9106 | ACC 0.7371
TRAIN: EPOCH 0002/0005 | BATCH 0029/0468 | LOSS: 0.9131 | ACC 0.7365
TRAIN: EPOCH 0002/0005 | BATCH 0030/0468 | LOSS: 0.9095 | ACC 0.7361
TRAIN: EPOCH 0002/0005 | BATCH 003

...

TRAIN: EPOCH 0004/0005 | BATCH 0419/0468 | LOSS: 0.1685 | ACC 0.9507
TRAIN: EPOCH 0004/0005 | BATCH 0420/0468 | LOSS: 0.1682 | ACC 0.9507
TRAIN: EPOCH 0004/0005 | BATCH 0421/0468 | LOSS: 0.1684 | ACC 0.9506
TRAIN: EPOCH 0004/0005 | BATCH 0422/0468 | LOSS: 0.1684 | ACC 0.9507
TRAIN: EPOCH 0004/0005 | BATCH 0423/0468 | LOSS: 0.1683 | ACC 0.9507
TRAIN: EPOCH 0004/0005 | BATCH 0424/0468 | LOSS: 0.1685 | ACC 0.9507
TRAIN: EPOCH 0004/0005 | BATCH 0425/0468 | LOSS: 0.1685 | ACC 0.9506
TRAIN: EPOCH 0004/0005 | BATCH 0426/0468 | LOSS: 0.1685 | ACC 0.9507
TRAIN: EPOCH 0004/0005 | BATCH 0427/0468 | LOSS: 0.1685 | ACC 0.9506
TRAIN: EPOCH 0004/0005 | BATCH 0428/0468 | LOSS: 0.1685 | ACC 0.9506
TRAIN: EPOCH 0004/0005 | BATCH 0429/0468 | LOSS: 0.1685 | ACC 0.9505
TRAIN: EPOCH 0004/0005 | BATCH 0430/0468 | LOSS: 0.1683 | ACC 0.9506
TRAIN: EPOCH 0004/0005 | BATCH 0431/0468 | LOSS: 0.1683 | ACC 0.9507
TRAIN: EPOCH 0004/0005 | BATCH 0432/0468 | LOSS: 0.1684 | ACC 0.9506
TRAIN: EPOCH 0004/0005 | BATCH 043

TRAIN: EPOCH 0005/0005 | BATCH 0071/0468 | LOSS: 0.1424 | ACC 0.9576
TRAIN: EPOCH 0005/0005 | BATCH 0072/0468 | LOSS: 0.1430 | ACC 0.9573
TRAIN: EPOCH 0005/0005 | BATCH 0073/0468 | LOSS: 0.1417 | ACC 0.9578
TRAIN: EPOCH 0005/0005 | BATCH 0074/0468 | LOSS: 0.1417 | ACC 0.9579
TRAIN: EPOCH 0005/0005 | BATCH 0075/0468 | LOSS: 0.1424 | ACC 0.9578
TRAIN: EPOCH 0005/0005 | BATCH 0076/0468 | LOSS: 0.1418 | ACC 0.9579
TRAIN: EPOCH 0005/0005 | BATCH 0077/0468 | LOSS: 0.1410 | ACC 0.9582
TRAIN: EPOCH 0005/0005 | BATCH 0078/0468 | LOSS: 0.1404 | ACC 0.9585
TRAIN: EPOCH 0005/0005 | BATCH 0079/0468 | LOSS: 0.1398 | ACC 0.9587
TRAIN: EPOCH 0005/0005 | BATCH 0080/0468 | LOSS: 0.1397 | ACC 0.9588
TRAIN: EPOCH 0005/0005 | BATCH 0081/0468 | LOSS: 0.1403 | ACC 0.9587
TRAIN: EPOCH 0005/0005 | BATCH 0082/0468 | LOSS: 0.1403 | ACC 0.9588
TRAIN: EPOCH 0005/0005 | BATCH 0083/0468 | LOSS: 0.1397 | ACC 0.9589
TRAIN: EPOCH 0005/0005 | BATCH 0084/0468 | LOSS: 0.1400 | ACC 0.9586
TRAIN: EPOCH 0005/0005 | BATCH 008

TRAIN: EPOCH 0005/0005 | BATCH 0194/0468 | LOSS: 0.1423 | ACC 0.9584
TRAIN: EPOCH 0005/0005 | BATCH 0195/0468 | LOSS: 0.1424 | ACC 0.9584
TRAIN: EPOCH 0005/0005 | BATCH 0196/0468 | LOSS: 0.1426 | ACC 0.9584
TRAIN: EPOCH 0005/0005 | BATCH 0197/0468 | LOSS: 0.1429 | ACC 0.9582
TRAIN: EPOCH 0005/0005 | BATCH 0198/0468 | LOSS: 0.1426 | ACC 0.9583
TRAIN: EPOCH 0005/0005 | BATCH 0199/0468 | LOSS: 0.1425 | ACC 0.9582
TRAIN: EPOCH 0005/0005 | BATCH 0200/0468 | LOSS: 0.1430 | ACC 0.9581
TRAIN: EPOCH 0005/0005 | BATCH 0201/0468 | LOSS: 0.1428 | ACC 0.9582
TRAIN: EPOCH 0005/0005 | BATCH 0202/0468 | LOSS: 0.1427 | ACC 0.9582
TRAIN: EPOCH 0005/0005 | BATCH 0203/0468 | LOSS: 0.1425 | ACC 0.9583
TRAIN: EPOCH 0005/0005 | BATCH 0204/0468 | LOSS: 0.1420 | ACC 0.9585
TRAIN: EPOCH 0005/0005 | BATCH 0205/0468 | LOSS: 0.1421 | ACC 0.9584
TRAIN: EPOCH 0005/0005 | BATCH 0206/0468 | LOSS: 0.1422 | ACC 0.9584
TRAIN: EPOCH 0005/0005 | BATCH 0207/0468 | LOSS: 0.1421 | ACC 0.9583
TRAIN: EPOCH 0005/0005 | BATCH 020

TRAIN: EPOCH 0005/0005 | BATCH 0319/0468 | LOSS: 0.1425 | ACC 0.9579
TRAIN: EPOCH 0005/0005 | BATCH 0320/0468 | LOSS: 0.1424 | ACC 0.9579
TRAIN: EPOCH 0005/0005 | BATCH 0321/0468 | LOSS: 0.1425 | ACC 0.9579
TRAIN: EPOCH 0005/0005 | BATCH 0322/0468 | LOSS: 0.1423 | ACC 0.9580
TRAIN: EPOCH 0005/0005 | BATCH 0323/0468 | LOSS: 0.1425 | ACC 0.9579
TRAIN: EPOCH 0005/0005 | BATCH 0324/0468 | LOSS: 0.1425 | ACC 0.9579
TRAIN: EPOCH 0005/0005 | BATCH 0325/0468 | LOSS: 0.1422 | ACC 0.9580
TRAIN: EPOCH 0005/0005 | BATCH 0326/0468 | LOSS: 0.1421 | ACC 0.9580
TRAIN: EPOCH 0005/0005 | BATCH 0327/0468 | LOSS: 0.1421 | ACC 0.9580
TRAIN: EPOCH 0005/0005 | BATCH 0328/0468 | LOSS: 0.1420 | ACC 0.9581
TRAIN: EPOCH 0005/0005 | BATCH 0329/0468 | LOSS: 0.1419 | ACC 0.9581
TRAIN: EPOCH 0005/0005 | BATCH 0330/0468 | LOSS: 0.1417 | ACC 0.9582
TRAIN: EPOCH 0005/0005 | BATCH 0331/0468 | LOSS: 0.1419 | ACC 0.9581
TRAIN: EPOCH 0005/0005 | BATCH 0332/0468 | LOSS: 0.1419 | ACC 0.9581
TRAIN: EPOCH 0005/0005 | BATCH 033

TRAIN: EPOCH 0005/0005 | BATCH 0438/0468 | LOSS: 0.1374 | ACC 0.9592
TRAIN: EPOCH 0005/0005 | BATCH 0439/0468 | LOSS: 0.1372 | ACC 0.9592
TRAIN: EPOCH 0005/0005 | BATCH 0440/0468 | LOSS: 0.1372 | ACC 0.9593
TRAIN: EPOCH 0005/0005 | BATCH 0441/0468 | LOSS: 0.1373 | ACC 0.9593
TRAIN: EPOCH 0005/0005 | BATCH 0442/0468 | LOSS: 0.1374 | ACC 0.9592
TRAIN: EPOCH 0005/0005 | BATCH 0443/0468 | LOSS: 0.1373 | ACC 0.9593
TRAIN: EPOCH 0005/0005 | BATCH 0444/0468 | LOSS: 0.1374 | ACC 0.9592
TRAIN: EPOCH 0005/0005 | BATCH 0445/0468 | LOSS: 0.1373 | ACC 0.9593
TRAIN: EPOCH 0005/0005 | BATCH 0446/0468 | LOSS: 0.1374 | ACC 0.9593
TRAIN: EPOCH 0005/0005 | BATCH 0447/0468 | LOSS: 0.1373 | ACC 0.9593
TRAIN: EPOCH 0005/0005 | BATCH 0448/0468 | LOSS: 0.1373 | ACC 0.9593
TRAIN: EPOCH 0005/0005 | BATCH 0449/0468 | LOSS: 0.1371 | ACC 0.9594
TRAIN: EPOCH 0005/0005 | BATCH 0450/0468 | LOSS: 0.1371 | ACC 0.9594
TRAIN: EPOCH 0005/0005 | BATCH 0451/0468 | LOSS: 0.1370 | ACC 0.9594
TRAIN: EPOCH 0005/0005 | BATCH 045